In [1]:
import os, subprocess, re, multiprocess

In [2]:
def run(cmd):    
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [ b.decode('UTF-8') for b in process.communicate() ]
    
    if error.strip() != "":
        print(error)

    print(out)

def __do_runNSave(cmd, out_path, ret_times, get_times_eye=True):
     # result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [ b.decode('UTF-8') for b in process.communicate() ]
    out = out.rstrip()
    # print("out:", out)
    # print("error:", error)
        
    with open(out_path, 'w') as fh:
        fh.write(out)
    
    if "** ERROR **" in error:
        print("ERROR:", error)
    
    elif get_times_eye:
        netw_time = int(re.search("networking \d+ \[msec cputime\] (\d+) \[msec walltime\]", error).group(1))
        reas_time = int(re.search("reasoning \d+ \[msec cputime\] (\d+) \[msec walltime\]", error).group(1))
        ret_times.append(netw_time); ret_times.append(reas_time)
        # return netw_time, reas_time

def runNSave_timed(cmd, out_path, max_time, get_times_eye=True):
    mngr = multiprocess.Manager()
    ret_times = mngr.list()
    
    p = multiprocess.Process(target=__do_runNSave, args=(cmd, out_path, ret_times, get_times_eye))
    p.start()
    
    p.join(max_time)
    if p.is_alive():
        p.kill()
        return (-1, -1)
    else:
        return ret_times
        
def runNSave(cmd, out_path, get_times_eye=True):
   ret_times = []
   __do_runNSave(cmd, out_path, ret_times, get_times_eye)
   
   return ret_times

def record_eye(times_file, query, data, type, phase, netw_time, reas_time):
    times_file.write(f"{query},{data},{type},{phase},{netw_time},{reas_time}\n")
    
def record_sparql(times_file, query, data, exec_time):
    times_file.write(f"{query},{data},{exec_time}\n")

## Convert SPARQL PP into N3 PP

In [ ]:
# NOTE for now run with Python 3.11.5 (manually fixed issue with NegatedPropertySet & reverse paths)

from rdflib.plugins.sparql import parser
from convert_pp_sparql_n3 import To_N3_Visitor

In [ ]:
# if someone else is running this notebook:

# @rdflib.plugins.sparql.parserutils.py#279
# add:
# elif isinstance(t,CompValue) or isinstance(t,URIRef):
#     res['part'] = t

### Convert

In [ ]:
# convert single query

# query = "SELECT ?x WHERE { ?x (:p1/:p2)* ?z ; !(:p3|:p4|:p5) ?a }"
# query = "PREFIX : <http://example.org/gmark/> " + \
#     "SELECT * WHERE { ?x0 !(^:p1|:p2) ?x3 } "
#     # "SELECT * WHERE { ?x0 ((^:p1/:p2*)?/:p3)+ ?x3 } "
#     # "SELECT * WHERE { ?x0 !(:p1|^:p2|:p3) ?x3 }"
query = """PREFIX : <http://example.org/gmark/> 
SELECT DISTINCT ?x2 ?x1 ?x3 ?x0 
WHERE {  {  ?x0 (((^:p1/:p1/^:p1/:p1)))* ?x1 . ?x1 ((^:p1/:p1/^:p1)|(^:p1/:p1/^:p1)) ?x2 . ?x2 (((:p1/^:p1)))+ ?x3 . } }
"""

query = parser.parseQuery(query)
query = query[1]

print(To_N3_Visitor().convert(query))

In [ ]:
# convert query folder

import os

visitor = To_N3_Visitor()

path = "../../gmark_50_new/mix"
files = list(os.listdir(path))
files.sort()
for file in files:
    if not file.endswith(".sparql"):
        continue
    print(file)
    with open(os.path.join(path, file), 'r') as fh:
        query = fh.read()
        query = parser.parseQuery(query)
        query = query[1]
        
        conv = visitor.convert(query)
        conv = "@prefix : <http://example.org/gmark/> .\n\n" + conv
        # print(conv)
        
        n3_file = file[0:file.index(".")] + ".n3"
        with open(os.path.join(path, "n3", n3_file), 'w') as fh2:
            fh2.write(conv)
        
        # print("\n")

## Run SPARQL PP

In [ ]:
from resCSV2N3 import convert as csv2n3
# from rdflib import Graph

In [ ]:
def exec_sparql(query_file, data_file, result_file):
    process = subprocess.Popen(['java', '-jar', "../test/run/sparql.jar", "-n3", data_file, "-query",
                               query_file, "-result", result_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [b.decode('UTF-8') for b in process.communicate()]
    if error.strip() != "":
        print(error)
    else:
        time = re.search("time: (.*)", out).group(1)
        return time

    # g = Graph(); g.parse(data_file)
    # g.query(open(query_file, 'r').read())

In [ ]:
# run single query

# path = "../../gmark_50_new/mix"
path = "test"
file = "test3.sparql"
name = file[:file.index(".")]
query_file = os.path.join(path, file)
data_file = os.path.join(path, "data3.n3")
result_file_csv = os.path.join(path, f"{name}_results_sparql.csv")

exec_time = exec_sparql(query_file, data_file, result_file_csv)
print(exec_time)

In [ ]:
# run query folder

path = "../../gmark_50_new/mix"

times_file = open(os.path.join(path, "results", "times.csv"), 'w')

files = list(os.listdir(path))
files.sort()
for file in files:
    if not file.endswith(".sparql"):
        continue
    print(file)
    
    name = file[:file.index(".")]
    query_file = os.path.join(path, file)
    data_file = "data.n3"
    data_path = os.path.join(path, "data.n3")
    result_path_csv = os.path.join(path, "results", f"{name}.csv")
    result_path_n3 = os.path.join(path, "results", f"{name}.n3")
    
    exec_time = exec_sparql(query_file, data_path, result_path_csv)
    record_sparql(times_file, file, data_file, exec_time)
    
    csv2n3(file=result_path_csv , ordered=False, out=result_path_n3)

## Run N3 PP

### Ground & normalize

In [ ]:
# normalize single file

path = "../../gmark_50_new/mix/n3"
file = "query-6.n3"
or_file = os.path.join(path, file)
norm_file = os.path.join(path, "normalized", file)

# ground
runNSave(["eye", "--quiet", or_file, "--no-qvars", "--nope", "--pass-all"], norm_file, get_times_eye=False)

# normalize
runNSave(["eye", "--quiet", norm_file, "aux_list.n3", "--query", "list-predicate.n3", "--quantify", "http://www.w3.org/2000/10/swap/var#", "--nope"], norm_file, get_times_eye=False)

In [ ]:
# normalize entire folder

path = "../../gmark_50_new/mix"
for file in os.listdir(path):    
    if not file.endswith(".sparql"):
        continue
    
    file = file[:file.index(".")] + ".n3"
    or_file = os.path.join(path, "n3", file)
    norm_file = os.path.join(path, "n3", "normalized", file)
    
    print(file)
    
    # ground
    runNSave(["eye", "--quiet", or_file, "--no-qvars", "--nope", "--pass-all"], norm_file, get_times_eye=False)
    
    # normalize
    runNSave(["eye", "--quiet", norm_file, "aux_list.n3", "--query", "list-predicate.n3", "--quantify", "http://www.w3.org/2000/10/swap/var#", "--nope"], norm_file, get_times_eye=False)

### Run

In [ ]:
# run single rule

path = "../../gmark_50_new/mix"
file = "query-6.n3"
norm_file = os.path.join(path, "n3", "normalized", file)
data_path = os.path.join(path, "data.n3")

res_file = os.path.join(path, "n3", "results", "direct", file)
netw_time, reas_time = runNSave_timed(["eye", data_path, "property-paths-direct.n3", "--query", norm_file, "--nope"], res_file, max_time=5)
print(netw_time, reas_time)

# rules_file = os.path.join(path, "test3_rule_creation.n3")
# netw_time, reas_time = runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation.n3", "--nope"], rules_file)
# netw_time2, reas_time2 = runNSave(["eye", data_path, rules_file, "--query", norm_file, "--nope"], res_file)

# data-red1: instantaneous
# data-red2: 8s
# data-red3: 16s
# data-red4: > 2min


In [3]:
# run folder

path = "../../gmark_50_new/mix"

times_file = open(os.path.join(path, "n3", "results", "times.csv"), 'w')
times_file.write("query,data,type,phase,netw_time,reas_time\n")

files = list(os.listdir(os.path.join(path, "n3", "normalized")))
files.sort()
for file in files:
    if not file.startswith("query") and not file.endswith(".n3"):
        continue
    
    name = file[:file.index(".")]
    norm_file = os.path.join(path, "n3", "normalized", file)
    data_file = "data.n3"
    data_path = os.path.join(path, data_file)
    dir_res_file = os.path.join(path, "n3", "results", "direct", file)
    bwd_res_file = os.path.join(path, "n3", "results", "bwd", file)
    fwd_res_file = os.path.join(path, "n3", "results", "fwd", file)
    
    if not os.path.exists(norm_file):
        continue
    
    print(norm_file)
    
    max_time = 60
    
    try:
        # - direct
        netw_time, reas_time = runNSave_timed(["eye", data_path, "property-paths-direct.n3", "--query", norm_file, "--nope"], dir_res_file, max_time=max_time)
        record_eye(times_file, file, data_file, 'direct', 'n/a', netw_time, reas_time)
        
        # - backward
        tmp_file = os.path.join(path, "n3", "gen", f"{name}_bwd.n3")
        netw_time1, reas_time1 = runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation-backwards.n3", "--nope"], tmp_file)
        record_eye(times_file, file, data_file, 'bwd', 'create', netw_time1, reas_time1)
        
        netw_time2, reas_time2 = runNSave_timed(["eye", data_path, tmp_file, "--query", norm_file, "--nope"], bwd_res_file, max_time=max_time)
        record_eye(times_file, file, data_file, 'bwd', 'run', netw_time2, reas_time2)
        record_eye(times_file, file, data_file, 'bwd', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
            
        # - forward
        tmp_file = os.path.join(path, "n3", "gen", f"{name}_fwd.n3")
        netw_time1, reas_time1 = runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation.n3", "--nope"], tmp_file)
        record_eye(times_file, file, data_file, 'fwd', 'create', netw_time1, reas_time1)
        netw_time2, reas_time2 = runNSave_timed(["eye", data_path, tmp_file, "--query", norm_file, "--nope"], fwd_res_file, max_time=max_time)
        record_eye(times_file, file, data_file, 'fwd', 'run', netw_time2, reas_time2)
        record_eye(times_file, file, data_file, 'fwd', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
    
    except:
        print(">>error<<")
    
    times_file.flush()

../../gmark_50_new/mix/n3/normalized/query-0.n3
../../gmark_50_new/mix/n3/normalized/query-1.n3
../../gmark_50_new/mix/n3/normalized/query-10.n3
../../gmark_50_new/mix/n3/normalized/query-11.n3
../../gmark_50_new/mix/n3/normalized/query-12.n3
../../gmark_50_new/mix/n3/normalized/query-13.n3
../../gmark_50_new/mix/n3/normalized/query-14.n3
../../gmark_50_new/mix/n3/normalized/query-15.n3
../../gmark_50_new/mix/n3/normalized/query-16.n3
../../gmark_50_new/mix/n3/normalized/query-17.n3
../../gmark_50_new/mix/n3/normalized/query-18.n3
../../gmark_50_new/mix/n3/normalized/query-19.n3
../../gmark_50_new/mix/n3/normalized/query-2.n3
../../gmark_50_new/mix/n3/normalized/query-20.n3
../../gmark_50_new/mix/n3/normalized/query-21.n3
../../gmark_50_new/mix/n3/normalized/query-22.n3
../../gmark_50_new/mix/n3/normalized/query-23.n3
../../gmark_50_new/mix/n3/normalized/query-24.n3
../../gmark_50_new/mix/n3/normalized/query-25.n3
../../gmark_50_new/mix/n3/normalized/query-26.n3
../../gmark_50_new/mix/

## Run SparqLog PP on Nemo

In [ ]:
path = "test"
in_path = os.path.join(path, "misc/test3.sparql")
out_path = os.path.join(path, "misc/test3_sparqlog.nmo")

run(["java", "-jar", "sparql2sparqlog.jar", in_path, out_path]) # translate
run(["/opt/nemo-0.6.0/nmo", out_path, "--export-dir", "results", "--overwrite-results"]) # run

## Run N3 PP on Nemo

In [ ]:
path = "test"
# substitute for rule file without collections
in_path = os.path.join(path, "data.n3") 
out_path = os.path.join(path, "data.nmo")

run(["java", "-jar", "n32nmo.jar", in_path, out_path]) # translate
# run(["/opt/nemo-0.6.0/nmo", out_path, "--export-dir", "results", "--overwrite-results"]) # run

## Compare results

In [4]:
import logging, sys

def msg(test, msg, to):
    str = f"- {test}: {msg}"
    log(str, to)
    
def log(str, to):
    to.write(str + "\n")
    print(str)
    to.flush()

In [5]:
def compare_results(file1, file2, test, to):
    process = subprocess.Popen(['java', '-jar', "compare_res.jar", file1, file2], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [ b.decode('UTF-8') for b in process.communicate() ]
    out = out.strip()
    
    if out != "results are the same!":
        msg(test, out, to=to)

In [6]:
path1 = "../../gmark_50_new/mix/results"
files = list(os.listdir(path1))
files.sort()

for version in ('direct', 'fwd', 'bwd'):
    with open(f"cmp_{version}_out.txt", 'w') as logfile:
        log(f"> comparing '{version}'", to=logfile)
        
        for file in files:
            if not file.endswith(".n3"):
                continue
            
            print(file)
            file1 = os.path.join(path1, file)
            
            path2 = f"../../gmark_50_new/mix/n3/results/{version}"
            file2 = os.path.join(path2, file)
            if not os.path.isfile(file2):
                msg(file, "cannot find file2", to=logfile)
            else:
                compare_results(file1, file2, test=file, to=logfile)

> comparing 'direct'
query-0.n3
query-1.n3
query-10.n3
query-11.n3
query-12.n3
query-13.n3
query-14.n3
- query-14.n3: cannot find file2
query-15.n3
query-16.n3
query-17.n3
query-18.n3
query-19.n3
query-2.n3
query-20.n3
query-21.n3
query-22.n3
query-23.n3
query-24.n3
query-25.n3
query-26.n3
query-27.n3
- query-27.n3: cannot find file2
query-28.n3
- query-28.n3: cannot find file2
query-29.n3
query-3.n3
query-30.n3
query-31.n3
query-32.n3
query-33.n3
query-34.n3
- query-34.n3: cannot find file2
query-35.n3
query-36.n3
query-37.n3
query-38.n3
- query-38.n3: cannot find file2
query-39.n3
query-4.n3
query-40.n3
- query-40.n3: cannot find file2
query-41.n3
query-42.n3
query-43.n3
- query-43.n3: cannot find file2
query-44.n3
query-45.n3
- query-45.n3: cannot find file2
query-46.n3
query-47.n3
query-48.n3
query-49.n3
- query-49.n3: cannot find file2
query-5.n3
query-6.n3
- query-6.n3: cannot find file2
query-7.n3
query-8.n3
- query-8.n3: cannot find file2
query-9.n3
> comparing 'fwd'
query-0.n3